## 问题定义

本问题为求解二维流动的传热，涉及到平流-扩散传输、NS方程。

本问题的几何定义如下：
![problem](./resource/problem.png)

矩形求解域中有三个散热鳍片。鳍片的温度维持在350K。左侧为入口，右侧为出口。入口流体温度为293.498K。矩形求解域以及鳍片参数如图所示。假设输入速度为抛物线速度剖面，峰值为1.5m/s。运动粘度为0.01$m^2/s$，Prandtl数为5.

注意虽然在设定条件下为应当为层流，但是本案例依然使用了零方程湍流模型（需要额外求解nu）。


### 求解目标

求解压力场、速度场、温度场以及nu

## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [19]:
import torch
import numpy as np
from sympy import Symbol, Eq

import modulus.sym
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.hydra.utils import compose
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.sym.utils.sympy.functions import parabola
from modulus.sym.utils.io import csv_to_dict, ValidatorPlotter
from modulus.sym.eq.pdes.navier_stokes import NavierStokes, GradNormal
from modulus.sym.eq.pdes.basic import NormalDotVec
from modulus.sym.eq.pdes.turbulence_zero_eq import ZeroEquation
from modulus.sym.eq.pdes.advection_diffusion import AdvectionDiffusion
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)
from modulus.sym.domain.monitor import PointwiseMonitor
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.geometry import Parameterization, Parameter

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 500000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: 1000
  rec_inference_freq: 1000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: false
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:
  fully_connected:
    arch_type: fully_connected
    input_keys: ???

### 定义必要组件

#### 常数

In [4]:
channel_length = (-2.5, 2.5)  # channel长度
channel_width = (-0.5, 0.5)  # channel宽度
heat_sink_origin = (-1, -0.3)  # 散热鳍片起始位置
nr_heat_sink_fins = 3  # 散热鳍片数量
gap = 0.15 + 0.1  # 鳍片间距 + 鳍片宽度
heat_sink_length = 1.0  # 散热鳍片长度
heat_sink_fin_thickness = 0.1  # 散热鳍片厚度
inlet_vel = 1.5  # 入口峰值速度
heat_sink_temp = 350  # 散热鳍片温度
base_temp = 293.498  # 入口流体温度
nu = 0.01  # 基础nu
diffusivity = 0.01 / 5  # 热扩散系数： Pr = nu/diffusivity

In [5]:
x, y = Symbol("x"), Symbol("y")

#### Geo

In [6]:
# channel 
channel = Channel2D(
    (channel_length[0], channel_width[0]), (channel_length[1], channel_width[1])
)

# 鳍片位置
# 第一个鳍片
heat_sink = Rectangle(
    heat_sink_origin,
    (
        heat_sink_origin[0] + heat_sink_length,
        heat_sink_origin[1] + heat_sink_fin_thickness,
    ),
)
# 后续鳍片
for i in range(1, nr_heat_sink_fins):
    heat_sink_origin = (heat_sink_origin[0], heat_sink_origin[1] + gap)
    fin = Rectangle(
        heat_sink_origin,
        (
            heat_sink_origin[0] + heat_sink_length,
            heat_sink_origin[1] + heat_sink_fin_thickness,
        ),
    )
    heat_sink = heat_sink + fin
    
# 求解区域（排除掉鳍片区域）
geo = channel - heat_sink

# 入口
inlet = Line(
    (channel_length[0], channel_width[0]), (channel_length[0], channel_width[1]), -1
)
# 出口
outlet = Line(
    (channel_length[1], channel_width[0]), (channel_length[1], channel_width[1]), 1
)

# 用于构造积分边界条件所需的内部平面（intermediate planes）
# 基本就是在channel_length范围内随机选择x_pos，然后构造y方向的竖线
x_pos = Parameter("x_pos")
integral_line = Line(
    (x_pos, channel_width[0]),
    (x_pos, channel_width[1]),
    1,
    parameterization=Parameterization({x_pos: channel_length}),
)

#### PDE

分别定义零方程和NS方程

In [7]:
# class ZeroEquation(PDE):
#     """
#     Zero Equation Turbulence model

#     Parameters
#     ==========
#     nu : float
#         The kinematic viscosity of the fluid.
#     max_distance : float
#         The maximum wall distance in the flow field.
#     rho : float, Sympy Symbol/Expr, str
#         The density. If `rho` is a str then it is
#         converted to Sympy Function of form 'rho(x,y,z,t)'.
#         If 'rho' is a Sympy Symbol or Expression then this
#         is substituted into the equation. Default is 1.
#     dim : int
#         Dimension of the Zero Equation Turbulence model (2 or 3).
#         Default is 3.
#     time : bool
#         If time-dependent equations or not. Default is True.

#     Example
#     ========
#     >>> zeroEq = ZeroEquation(nu=0.1, max_distance=2.0, dim=2)
#     >>> kEp.pprint()
#       nu: sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)
#       *Min(0.18, 0.419*normal_distance)**2 + 0.1
#     """

#     name = "ZeroEquation"

#     def __init__(
#         self, nu, max_distance, rho=1, dim=3, time=True
#     ):  # TODO add density into model
#         # set params
#         self.dim = dim
#         self.time = time

#         # model coefficients
#         self.max_distance = max_distance  # 最大长度
#         self.karman_constant = 0.419  # 3式的第一个常量
#         self.max_distance_ratio = 0.09  # 3式的第二个常量

#         # coordinates
#         x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

#         # time
#         t = Symbol("t")

#         # make input variables
#         input_variables = {"x": x, "y": y, "z": z, "t": t}
#         if self.dim == 2:
#             input_variables.pop("z")
#         if not self.time:
#             input_variables.pop("t")

#         # velocity componets
#         u = Function("u")(*input_variables)
#         v = Function("v")(*input_variables)
#         if self.dim == 3:
#             w = Function("w")(*input_variables)
#         else:
#             w = Number(0)

#         # density
#         if type(rho) is str:
#             rho = Function(rho)(*input_variables)
#         elif type(rho) in [float, int]:
#             rho = Number(rho)

#         # wall distance
#         normal_distance = Function("sdf")(*input_variables)

#         # mixing length
#         # 混合长度的计算式
#         mixing_length = Min(
#             self.karman_constant * normal_distance,
#             self.max_distance_ratio * self.max_distance,
#         )
        
#         # 计算G
#         G = (
#             2 * u.diff(x) ** 2
#             + 2 * v.diff(y) ** 2
#             + 2 * w.diff(z) ** 2
#             + (u.diff(y) + v.diff(x)) ** 2
#             + (u.diff(z) + w.diff(x)) ** 2
#             + (v.diff(z) + w.diff(y)) ** 2
#         )

#         # set equations
#         self.equations = {}
#         self.equations["nu"] = nu + rho * mixing_length**2 * sqrt(G)  # 创建nu关系式

In [8]:
# 零方程
ze = ZeroEquation(
        nu=nu, rho=1.0, dim=2, max_distance=(channel_width[1] - channel_width[0]) / 2
    )
print(ze.pprint())
# NS方程
# nu是可学习的
ns = NavierStokes(nu=ze.equations["nu"], rho=1.0, dim=2, time=False)
print(ns.pprint())
# 平流-扩散方程
ade = AdvectionDiffusion(T="c", rho=1.0, D=diffusivity, dim=2, time=False)
print(ade.pprint())
# 边界热交换约束
gn_c = GradNormal("c", dim=2, time=False)
print(gn_c.pprint())
# 积分连续性约束
normal_dot_vel = NormalDotVec(["u", "v"])
print(normal_dot_vel.pprint())

nu: 1.0*sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)*Min(0.045, 0.419*sdf)**2 + 0.01
None
continuity: 1.0*u__x + 1.0*v__y
momentum_x: -(1.0*sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)*Min(0.045, 0.419*sdf)**2 + 0.01)*u__x__x - (1.0*sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)*Min(0.045, 0.419*sdf)**2 + 0.01)*u__y__y - (1.0*((u__y + v__x)*(2*u__y__y + 2*v__x__y)/2 + 2*u__x*u__x__y + 2*v__y*v__y__y)*Min(0.045, 0.419*sdf)**2/sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2) + 0.838*sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)*Heaviside(0.045 - 0.419*sdf)*sdf__y*Min(0.045, 0.419*sdf))*u__y - (1.0*((u__y + v__x)*(2*u__y__y + 2*v__x__y)/2 + 2*u__x*u__x__y + 2*v__y*v__y__y)*Min(0.045, 0.419*sdf)**2/sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2) + 0.838*sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)*Heaviside(0.045 - 0.419*sdf)*sdf__y*Min(0.045, 0.419*sdf))*v__x - 2*(1.0*((u__y + v__x)*(2*v__x__x + 2*u__x__y)/2 + 2*u__x*u__x__x + 2*v__y*v__x__y)*Min(0.045, 0.419*sdf)**2/sqrt((u__y + v__x

#### Model

In [9]:
# 流体模型
# 输入为空间坐标
# 输出为u、v、p
flow_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    cfg=cfg.arch.fully_connected,
)
# 传热模型
# 输入为空间坐标
# 输出为温度c
# 注意这里是相对温度
# 计算方法为：(真实温度-入口基准温度) / 273.15
heat_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("c")],
    cfg=cfg.arch.fully_connected,
)
print(flow_net)

# 这个例子的node更复杂
# 首先是5个PDE
# 然后是两个模型
nodes = (
    ns.make_nodes()
    + ze.make_nodes()
    + ade.make_nodes(detach_names=["u", "v"])
    + gn_c.make_nodes()
    + normal_dot_vel.make_nodes()
    + [flow_net.make_node(name="flow_network")]
    + [heat_net.make_node(name="heat_network")]
)

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)


#### Domain

在Domain中定义约束以及训练所需的各种组件

In [10]:
# make domain
domain = Domain()

#### 入口约束

入口速度为峰值为1.5m/s的抛物面，因此使用parabola进行计算

In [11]:
inlet_parabola = parabola(
    y, inter_1=channel_width[0], inter_2=channel_width[1], height=inlet_vel
)
inlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=inlet,
    outvar={"u": inlet_parabola, "v": 0, "c": 0},
    batch_size=cfg.batch_size.inlet,
)
domain.add_constraint(inlet, "inlet")

#### 出口约束

压力出口
且规定压力出口的压强为0

In [12]:
# outlet
outlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=outlet,
    outvar={"p": 0},
    batch_size=cfg.batch_size.outlet,
)
domain.add_constraint(outlet, "outlet")

#### 鳍片表面约束

散热鳍片表面约束
规定速度为0，温度为鳍片温度

In [13]:
# heat_sink wall
hs_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=heat_sink,
    outvar={"u": 0, "v": 0, "c": (heat_sink_temp - base_temp) / 273.15},
    batch_size=cfg.batch_size.hs_wall,
)
domain.add_constraint(hs_wall, "heat_sink_wall")

#### 边界约束

在本问题中，定义边界的热交换为0，流速为0

In [14]:
# channel wall
channel_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=channel,
    outvar={"u": 0, "v": 0, "normal_gradient_c": 0},
    batch_size=cfg.batch_size.channel_wall,
)
domain.add_constraint(channel_wall, "channel_wall")

#### 初始条件

由于求解稳态解，因此无初始条件

#### 内部PDE和ZeroEq约束

In [15]:
# 流体约束
interior_flow = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.interior_flow,
    compute_sdf_derivatives=True,  # 以供零方程使用，计算sdf的导数，即l_m关于空间方向的导数
    lambda_weighting={
        "continuity": Symbol("sdf"),
        "momentum_x": Symbol("sdf"),
        "momentum_y": Symbol("sdf"),
    },
)
domain.add_constraint(interior_flow, "interior_flow")

In [16]:
# 平流扩散方程约束
# interior heat
interior_heat = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"advection_diffusion_c": 0},
    batch_size=cfg.batch_size.interior_heat,
    lambda_weighting={
        "advection_diffusion_c": 1.0,
    },
)
domain.add_constraint(interior_heat, "interior_heat")

#### 积分连续性约束

这是Modulus里的一个trick。

具体而言，当达到稳态时，流入多少就流出多少，这对于任何截面均成立。

那么就可以基于这个构造新的约束项了。

本问题中构造的约束是：基于流入的抛物面速度，对于任何y方向截面，其速度的积分总是固定值，这个固定值为1。

这一约束的有效性还需要其他材料支撑。

In [17]:
# integral continuity
def integral_criteria(invar, params):
    sdf = geo.sdf(invar, params)
    return np.greater(sdf["sdf"], 0)

integral_continuity = IntegralBoundaryConstraint(
    nodes=nodes,
    geometry=integral_line,
    outvar={"normal_dot_vel": 1},
    batch_size=cfg.batch_size.num_integral_continuity,
    integral_batch_size=cfg.batch_size.integral_continuity,
    lambda_weighting={"normal_dot_vel": 0.1},
    criteria=integral_criteria,  # 排除穿过鳍片的截面
)
domain.add_constraint(integral_continuity, "integral_continuity")

#### 验证器以及其他必要组件

In [21]:
class NSValidatorPlotter(ValidatorPlotter):
    "Define custom validator plotting class"

    def __call__(self, invar, true_outvar, pred_outvar):
        # only plot x,y dimensions
        # 确保仅绘制x和y，排除sdf
        invar = {k: v for k, v in invar.items() if k in ["x", "y"]}
        fs = super().__call__(invar, true_outvar, pred_outvar)
        return fs

In [23]:
# add validation data
file_path = "openfoam/heat_sink_zeroEq_Pr5_mesh20.csv"
if os.path.exists(to_absolute_path(file_path)):
    # 加载数据
    mapping = {
        "Points:0": "x",
        "Points:1": "y",
        "U:0": "u",
        "U:1": "v",
        "p": "p",
        "d": "sdf",
        "nuT": "nu",
        "T": "c",
    }
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["nu"] += nu
    openfoam_var["c"] += -base_temp
    openfoam_var["c"] /= 273.15
    openfoam_invar_numpy = {
        key: value
        for key, value in openfoam_var.items()
        if key in ["x", "y", "sdf"]
    }
    openfoam_outvar_numpy = {
        key: value
        for key, value in openfoam_var.items()
        if key in ["u", "v", "p", "c", "nu"]  # Removing "nu"
    }
    
    # 创建validator
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        plotter=NSValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

# monitors for force, residuals and temperature

# residual
global_monitor = PointwiseMonitor(
    geo.sample_interior(100),
    output_names=["continuity", "momentum_x", "momentum_y"],
    metrics={
        "mass_imbalance": lambda var: torch.sum(
            var["area"] * torch.abs(var["continuity"])
        ),
        "momentum_imbalance": lambda var: torch.sum(
            var["area"]
            * (torch.abs(var["momentum_x"]) + torch.abs(var["momentum_y"]))
        ),
    },
    nodes=nodes,
    requires_grad=True,
)
domain.add_monitor(global_monitor)

# force
force = PointwiseMonitor(
    heat_sink.sample_boundary(100),
    output_names=["p"],
    metrics={
        "force_x": lambda var: torch.sum(var["normal_x"] * var["area"] * var["p"]),
        "force_y": lambda var: torch.sum(var["normal_y"] * var["area"] * var["p"]),
    },
    nodes=nodes,
)
domain.add_monitor(force)

# peak T
peakT = PointwiseMonitor(
    heat_sink.sample_boundary(100),
    output_names=["c"],
    metrics={"peakT": lambda var: torch.max(var["c"])},
    nodes=nodes,
)
domain.add_monitor(peakT)

### 求解器以及求解

In [25]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [26]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [27]:
slv.solve()

2024-02-19 08:58:10,822 - modulus.sym.trainer - INFO - attempting to restore from: /workspace/06_2D_AdvectionDiffusion/outputs
2024-02-19 08:58:10,825 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-02-19 08:58:10,826 - modulus.sym.trainer - WARNING - model flow_network.0.pth not found
2024-02-19 08:58:10,828 - modulus.sym.trainer - WARNING - model heat_network.0.pth not found
2024-02-19 08:58:13,280 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  2.224e-01s
2024-02-19 08:58:18,953 - modulus.sym.trainer - INFO - [step:          0] record validators time:  5.671e+00s
2024-02-19 08:58:19,136 - modulus.sym.trainer - INFO - [step:          0] record monitor time:  1.570e-01s
2024-02-19 08:58:19,489 - modulus.sym.trainer - INFO - [step:          0] saved checkpoint to /workspace/06_2D_AdvectionDiffusion/outputs
2024-02-19 08:58:19,491 - modulus.sym.trainer - INFO - [step:          0] loss:  1.790e+00
2024-02-19 08:58:23,097 - modulus.sy

2024-02-19 09:05:11,120 - modulus.sym.trainer - INFO - [step:       4600] loss:  1.117e-01, time/iteration:  8.206e+01 ms
2024-02-19 09:05:19,336 - modulus.sym.trainer - INFO - [step:       4700] loss:  2.360e-01, time/iteration:  8.215e+01 ms
2024-02-19 09:05:27,468 - modulus.sym.trainer - INFO - [step:       4800] loss:  2.055e-01, time/iteration:  8.130e+01 ms
2024-02-19 09:05:35,508 - modulus.sym.trainer - INFO - [step:       4900] loss:  1.855e-01, time/iteration:  8.037e+01 ms
2024-02-19 09:05:51,481 - modulus.sym.trainer - INFO - [step:       5000] record validators time:  6.436e+00s
2024-02-19 09:05:51,585 - modulus.sym.trainer - INFO - [step:       5000] record monitor time:  1.005e-01s
2024-02-19 09:05:51,648 - modulus.sym.trainer - INFO - [step:       5000] saved checkpoint to /workspace/06_2D_AdvectionDiffusion/outputs
2024-02-19 09:05:51,651 - modulus.sym.trainer - INFO - [step:       5000] loss:  9.888e-02, time/iteration:  1.614e+02 ms
2024-02-19 09:06:00,079 - modulus.s

KeyboardInterrupt: 

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

如果使用了PointwiseValidator则可以直接查看验证的结果

![v](./outputs/validators/validator_v.png)

![u](./outputs/validators/validator_u.png)

![p](./outputs/validators/validator_p.png)

![c](./outputs/validators/validator_c.png)

![nu](./outputs/validators/validator_nu.png)